In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
import time

In [2]:
ept = pd.read_excel("ept_all.xlsx")
ept.head()

,ISO,Tanggal,Waktu,Ruangan,Pengawas,ID_Peserta,Hadir,NamaPeserta,TempatLahir,TanggalLahir,...,Versi,Kegiatan,Tahun,L1,G1,R1,L2,G2,R2,TotalSkor
0,581,2024-06-05,09.00 - 11.00,A,Halima,15386,True,M.Arief Abdillah,PONTIANAK,2002-04-20 00:00:00,...,7,Semester Akhir,2024,16.0,15.0,16.0,41.0,38.0,36.0,383.0
1,581,2024-06-05,09.00 - 11.00,A,Halima,15381,True,Fitri Rahmadani,PONTIANAK,2002-11-14 00:00:00,...,10,Semester Akhir,2024,13.0,10.0,15.0,38.0,33.0,35.0,353.0
2,581,2024-06-05,09.00 - 11.00,A,Halima,15384,False,Jesica Fitriani Silalahi,PONTIANAK,2002-07-27 00:00:00,...,5,Semester Akhir,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,581,2024-06-05,09.00 - 11.00,A,Halima,15383,True,Indri Oktavianti,SUNGAI RAYA,2001-06-29 00:00:00,...,4,Semester Akhir,2024,11.0,9.0,17.0,36.0,32.0,37.0,350.0
4,581,2024-06-05,09.00 - 11.00,A,Halima,15382,False,Fransiska Agnesia Rika,RUJAK DAMAI,2001-08-07 00:00:00,...,3,Semester Akhir,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def appendGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name('key.json', scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    existing_iso_values = worksheet.col_values(worksheet.find('ISO').col)  
    existing_iso_values = existing_iso_values[1:] 
    
    # Convert to set for faster comparison
    existing_iso_set = set(existing_iso_values)

    # Get unique ISO values from the DataFrame
    new_iso_values = set(df['ISO'].astype(str).unique())
    
    # Check for duplicates
    duplicates = existing_iso_set.intersection(new_iso_values)
    if duplicates:
        raise ValueError(f"Duplicate ISO values found: {duplicates}. Aborting the operation.")
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be appended
    data = df.values.tolist()
    
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        worksheet.append_rows(batch, value_input_option='RAW')
        time.sleep(1)

    


    

appendGsheet(df=ept, dari=588, sampai=591)

ValueError: Duplicate ISO values found: {'590', '588', '591', '589'}. Aborting the operation.

In [5]:
def updateGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = Credentials.from_service_account_file('key.json', scopes=scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be update
    data = df.values.tolist()
    
    # Update data in batches to reduce API calls
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        cell_range = f'A{i+15272}:AD{i+15272+len(batch)-1}'  # Adjust the range as needed
        worksheet.update(cell_range, batch)
        time.sleep(1)  # Add delay to avoid hitting rate limits
    
     

# 581 jangan diubah karena di mulai dari cell 15272
updateGsheet(df=ept, dari=581, sampai=591)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10324\3852440222.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10324\3852440222.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10324\3852440222.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]